In [1]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT

In [6]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))
image_dict = {'keto': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'ketogenic': 'https://theindianspot.com/wp-content/uploads/2017/09/KETO-DIET-ALL-YOU-WANT-TO-KNOW.jpg',
             'paleo': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
            'paleolithic': 'https://images.agoramedia.com/everydayhealth/gcms/Can-the-Paleo-Diet-Help-Diabetes-1440x810.jpg',
             'vegan': 'https://familydoctor.org/wp-content/uploads/2011/09/shutterstock_413417941-705x468.jpg',
             'vegetarian': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQo-azo8tn-BcljdjG3wp5Oub2vmC3OvDJ2-EXIhxB5dG3EJAq-'}

def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df = df.drop(['blog','byline','headline','keywords','multimedia','news_desk','print_page','score','type_of_material','uri'],axis =1)
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['expertise'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = image_dict[param]
    return df

def diet_NYT():
    empty = pd.DataFrame()
    for word in ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']:
        df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
        empty = empty.append(df, sort=True)
        print('Pulled '+word)
        time.sleep(2)
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

# def general_health():
#     empty = pd.DataFrame()
#     for page in [0,1]:
#         df = NYT_api_call_section_based('Health', 'The New York Times',page, last_week, today, nyt_api_key)
#         empty = empty.append(df)
#         time.sleep(2)
#     return empty

def intermediate_search():
    # df1 = general_health()
    df2 = diet_NYT()
    # frames = [df1,df2]
    # result = pd.concat(frames)
    return df2 #result

# Social Media

In [54]:
all_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst', '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX', '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda', '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs', '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

In [8]:
import tweepy

auth = tweepy.OAuthHandler('UGxquWoEO6bisCK6FGhBlrKYp', 'dRw1GVeyzgdfYQD1S4aEZRGaGCep9QusVZWI9gt4uHriOZG4WI')
auth.set_access_token('333368932-TP9Vp0cWTZ8FywrhtYj3kSvDjhcSPuISON7ROh4f', 'k42e1QG5t72HiDzi4M7WxRfTUvUodZLKFT6268Eb6mLzw')
api = tweepy.API(auth)

In [10]:
test = api.get_user('@DailyHealthTips')

In [20]:
trial = api.user_timeline('@DailyHealthTips')

In [67]:
trial[0].text

'Facial Flushing: Should You Worry If Your Face Turns Red When You Drink? #AskADoctor #DailyHealthTips… https://t.co/gamBbBOOkg'

In [49]:
trial[0].entities['hashtags'][0]['text']

'AskADoctor'

In [64]:
tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'), 
           'description': tweet.text, 'source':tweet.user.screen_name, 'formality': 'Informal'} for tweet in trial]

In [65]:
tweets[1]

{'id': 1061974494808629249,
 'date': '2018-11-12',
 'text': b'Largest Health Q&amp;A site. \nAsk a Doctor Online at \nhttps://t.co/kQBzB69Ql9 \n\n#oilyskin #moisturizer\xe2\x80\xa6 https://t.co/TIsWSmyUo3',
 'name': 'DailyHealthTips'}

In [ ]:
https://twitter.com/i/web/status/1060826864720248832

In [32]:
from IPython.core.display import HTML, Image

In [ ]:
display(HTML("<a href="+link+">"+source_name+': '+title+"</a>"))

In [29]:
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text


In [52]:
display(Tweet('https://twitter.com/DailyHealthTips/status/1062019789294579719'))
display(Tweet('https://twitter.com/DailyHealthTips/status/1061974494808629249'))

In [28]:
outtweets

[['1062019789294579719',
  datetime.datetime(2018, 11, 12, 16, 30, 23),
  b'Facial Flushing: Should You Worry If Your Face Turns Red When You Drink? #AskADoctor #DailyHealthTips\xe2\x80\xa6 https://t.co/gamBbBOOkg'],
 ['1061974494808629249',
  datetime.datetime(2018, 11, 12, 13, 30, 24),
  b'Largest Health Q&amp;A site. \nAsk a Doctor Online at \nhttps://t.co/kQBzB69Ql9 \n\n#oilyskin #moisturizer\xe2\x80\xa6 https://t.co/TIsWSmyUo3'],
 ['1061929159721697280',
  datetime.datetime(2018, 11, 12, 10, 30, 16),
  b'5 Tips for Driving When You Have Arthritis or Back Problems #AskADoctor #DailyHealthTips #HealthcareMagic \n\nhttps://t.co/6FY34TEDL3'],
 ['1061868710925873154',
  datetime.datetime(2018, 11, 12, 6, 30, 3),
  b'Largest Health Q&amp;A site. \nAsk a Doctor Online at \nhttps://t.co/kQBzB69Ql9 \n\n#obesity #AskADoctor #DailyHealthTips\xe2\x80\xa6 https://t.co/lbLUKkwgS2'],
 ['1061808322393849857',
  datetime.datetime(2018, 11, 12, 2, 30, 6),
  b'10 Anti-Aging Foods to Support Your 40s

In [12]:
test.keys()

AttributeError: 'User' object has no attribute 'keys'